In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers

df = pd.read_csv('dicindia.csv')

In [2]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-07-01,495.950012,495.950012,475.549988,480.649994,475.580719,6469
1,2021-07-02,482.700012,489.799988,478.000000,481.799988,476.718597,6278
2,2021-07-05,487.250000,500.000000,482.000000,494.299988,489.086761,12130
3,2021-07-06,487.250000,505.000000,487.250000,490.799988,485.623657,24211
4,2021-07-07,494.000000,494.049988,483.049988,485.149994,480.033264,6248
5,2021-07-08,485.149994,493.000000,478.700012,480.049988,474.987030,3489
6,2021-07-09,486.000000,489.600006,479.250000,484.549988,479.439606,5981
7,2021-07-12,490.000000,494.799988,483.250000,485.299988,480.181702,5107
8,2021-07-13,487.100006,495.799988,480.000000,493.700012,488.493103,17855
9,2021-07-14,499.000000,505.000000,491.000000,493.899994,488.690979,11459


In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
df.loc[:,'Close'] = scaler.fit_transform(np.array(df.loc[:,'Close']).reshape(-1,1))

In [4]:
def split_sequence(sequence, n_steps):
    X = []
    y = []
    for i in range(len(sequence)):
        end_idx = i + n_steps
        if end_idx > len(sequence) - 1:
            break
        Xtemp = sequence[i:end_idx]
        ytemp = sequence[end_idx]
        X.append(Xtemp)
        y.append(ytemp)
    
    return np.array(X), np.array(y)

In [5]:
train_len = int(len(df.loc[:,'Close']) * 0.7)
test_len = int(len(df.loc[:,'Close']) * 0.3)

train_data = df.loc[:train_len,'Close']
test_data = df.loc[train_len:,'Close']

In [6]:
timesteps = 7

X_train, y_train = split_sequence(np.array(train_data), timesteps)
X_test, y_test = split_sequence(np.array(test_data), timesteps)

In [7]:
n_features = 1
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], n_features)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], n_features)

model = Sequential()
model.add(LSTM(32, activation = 'relu', return_sequences=True, input_shape=(timesteps,n_features)))
# model.add(LSTM(32, activation = 'relu', return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss="mean_squared_error", metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 32)             4352      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 12705 (49.63 KB)
Trainable params: 12705 (49.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.fit(X_train, y_train, epochs = 100, batch_size=30)

Epoch 1/100


12/12 [==============================] - 5s 9ms/step - loss: 0.1391 - accuracy: 0.0029
Epoch 2/100
12/12 [==============================] - 0s 8ms/step - loss: 0.0425 - accuracy: 0.0029
Epoch 3/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0194 - accuracy: 0.0059
Epoch 4/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0147 - accuracy: 0.0059
Epoch 5/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0103 - accuracy: 0.0059
Epoch 6/100
12/12 [==============================] - 0s 10ms/step - loss: 0.0066 - accuracy: 0.0059
Epoch 7/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0050 - accuracy: 0.0059
Epoch 8/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0050 - accuracy: 0.0059
Epoch 9/100
12/12 [==============================] - 0s 10ms/step - loss: 0.0049 - accuracy: 0.0059
Epoch 10/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0048 - accuracy: 0.0059
Epoch 11/100
12/12 

In [9]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

5/5 [==============================] - 0s 4ms/step


In [10]:
import math
from sklearn.metrics import mean_squared_error

rmse_train = math.sqrt(mean_squared_error(y_train, train_predict))
rmse_test = math.sqrt(mean_squared_error(y_test, test_predict))
print(rmse_train)
print("\n")
print(rmse_test)

412.41269429909795


398.0698905663752
